In [1]:
import json
import numpy as np
import torch
from utils import get_pkl


In [2]:
train_dictionary = get_pkl(f"./data/pkls/train_queries.pkl")
train_queries = get_pkl(f"./data/pkls/train_queries.pkl")

query_names, query_ids = [row[0] for row in train_queries], [row[1] for row in train_queries]
dict_names, dict_ids = [row[0] for row in train_dictionary], [row[1] for row in train_dictionary]


In [3]:

from biosyn.biosyn import BioSyn


biosyn = BioSyn(25, False, 20, 'dmis-lab/biobert-base-cased-v1.1' )

   

c:\Users\antoi\Desktop\thesis\1_2_biosyn_replicate\git_repo_final\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ns = dict_names[:32]
embs = biosyn.embed_dense_optimized(ns, return_tensor=False)


In [6]:
embs.shape[1]

768

In [3]:
with open("./data/pkls/dicts_1.json") as f:
    meta_d = json.load(f)
with open("./data/pkls/queries_1.json") as f:
    meta_q = json.load(f)
with open("./data/pkls/results_1.json") as f:
    meta_r = json.load(f)


mm_d = np.memmap("./data/pkls/dicts_1.fp16.mmap", 
                 dtype=np.float16 if meta_d["dtype"] == "fp16" else np.float32, 
                 shape=(meta_d["N"], meta_d["d"]))


mm_q = np.memmap("./data/pkls/queries_1.fp16.mmap", 
                 dtype=np.float16 if meta_q["dtype"] == "fp16" else np.float32, 
                 shape=((meta_r["N"], meta_q["d"])))

mm_r = np.memmap("./data/pkls/results_1.mmap", 
                 dtype=np.float16 if meta_r["dtype"] == "fp16" else np.int32, 
                 shape=((meta_r["N"], meta_r["d"])))



In [30]:
dict_ids_sets = [set(s.split("|")) if isinstance(s, str) else set(s) for s in dict_ids]
query_id_tokens = [tuple(q.split("|")) if isinstance(q, str) else tuple(q) for q in query_ids]



In [36]:
cui_to_dict_idx = {}
for i, cui in enumerate(dict_ids):
    toks = cui.split("|") if isinstance(cui, str) else list(cui)
    for t in toks: cui_to_dict_idx.setdefault(t, []).append(i)


In [ ]:
ls = [1,3,6,7,25063,13,36013,36017]
[dict_names[idx] for idx in ls]


In [ ]:
gold_idx_per_query = []
labels_per_query = []
for q_idx, cand_idxs in enumerate(mm_r):
    q_id_tokens = query_id_tokens[q_idx]
    possible = set(cui_to_dict_idx.get(q_id_tokens[0], []))
    print(possible)
    
    if q_idx == 1:
        break

In [ ]:
from transformers import AutoTokenizer
from biosyn.dataloader import CandidateDataset


tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1' )
ds = CandidateDataset(queries=train_queries, dicts=train_dictionary, tokenizer=tokenizer, max_length=25, topk=20, pre_tokenize=True)

In [ ]:
for query_idx, query_cands in enumerate(mm_r):
    print(f"query_idx: {query_idx}")
    invalid = [id for id in query_cands if id > len(dict_ids)]
    print(f"invalid: {invalid}")

In [ ]:
# flat_arr = mm_r.flatten()
# valid_indexes = flat_arr[flat_arr < len(dict_ids)]
# len(valid_indexes)
# len(flat_arr)


In [55]:
import faiss
chunk = 12288

N, d = meta_d["N"], meta_d["d"]
index = faiss.IndexFlatIP(meta_d["d"])


#stream add to dict in chunks
for s in range(0, N, chunk):
    e = min(s+chunk, N)
    part = mm_d[s:e]
    t = part
    index.add(t)
    del t
faiss_index = index


In [ ]:
from utils import get_pkl

i_all = get_pkl("./data/pkls/I_all.pkl")

In [ ]:
len(dict_ids)

In [ ]:
mm_d.shape


In [ ]:
i_flat = i_all.flatten()
[idx for idx in i_flat if idx < len(dict_ids)]

In [1]:
import numpy as np 

shape = (3,4)

# np.empty(shape)

arr_1 = np.empty(shape=shape )

arr = np.memmap(shape=shape,mode="w+", filename="draft_mmap.mmap" )


In [ ]:
arr[0, 0] = 1

In [ ]:
import numpy as np

arr = np.memmap("draft_mmap.mmap", mode="r", dtype=np.int32, shape=(3,4) )
arr